In [1]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from recpack.preprocessing.preprocessors import DataFramePreprocessor
from recpack.preprocessing.filters import MinItemsPerUser, MinUsersPerItem
from recpack.algorithms import TARSItemKNN, ItemKNN
from recpack.scenarios import Timed
from recpack.matrix import InteractionMatrix

# import utils file from previous lecture
import sys
sys.path.append('../lecture4')
from utils import DATA_PATH
from evaluation import apk

# This file builds on the code in https://github.com/radekosmulski/personalized_fashion_recs/blob/main/03a_Basic_Model_Local_Validation.ipynb

In [2]:
transactions = pd.read_parquet(f'{DATA_PATH}/transactions_train.parquet')
customers = pd.read_parquet(f'{DATA_PATH}/customers.parquet')
articles = pd.read_parquet(f'{DATA_PATH}/articles.parquet')

In [3]:
test_week = transactions.week.max()
transactions = transactions[transactions.week != transactions.week.max()]
transactions = transactions[transactions.week > transactions.week.max() - 10]

# Generating candidates

### Last purchase candidates

In [4]:
%%time

c2weeks = transactions.groupby('customer_id')['week'].unique()

CPU times: user 8.01 s, sys: 81.1 ms, total: 8.09 s
Wall time: 8.1 s


In [5]:
%%time

c2weeks2shifted_weeks = {}

for c_id, weeks in c2weeks.items():
    c2weeks2shifted_weeks[c_id] = {}
    for i in range(weeks.shape[0]-1):
        c2weeks2shifted_weeks[c_id][weeks[i]] = weeks[i+1]
    c2weeks2shifted_weeks[c_id][weeks[-1]] = test_week

CPU times: user 259 ms, sys: 19.3 ms, total: 278 ms
Wall time: 277 ms


In [6]:
candidates_last_purchase = transactions.copy()

In [7]:
%%time

weeks = []
for i, (c_id, week) in enumerate(zip(transactions['customer_id'], transactions['week'])):
    weeks.append(c2weeks2shifted_weeks[c_id][week])
    
candidates_last_purchase.week=weeks

CPU times: user 9.12 s, sys: 74.1 ms, total: 9.19 s
Wall time: 9.19 s


### Bestsellers candidates

In [8]:
mean_price = transactions \
    .groupby(['week', 'article_id'])['price'].mean()

In [9]:
sales = transactions \
    .groupby('week')['article_id'].value_counts() \
    .groupby('week').rank(method='dense', ascending=False) \
    .groupby('week').head(12).rename('bestseller_rank').astype('int8')

In [10]:
bestsellers_previous_week = pd.merge(sales, mean_price, on=['week', 'article_id']).reset_index()
bestsellers_previous_week.week += 1

In [11]:
unique_transactions = transactions \
    .groupby(['week', 'customer_id']) \
    .head(1) \
    .drop(columns=['article_id', 'price']) \
    .copy()

In [12]:
candidates_bestsellers = pd.merge(
    unique_transactions,
    bestsellers_previous_week,
    on='week',
)

In [13]:
test_set_transactions = unique_transactions.drop_duplicates('customer_id').reset_index(drop=True)
test_set_transactions.week = test_week

### K most similar items

In [14]:
proc = DataFramePreprocessor(item_ix='article_id', user_ix='customer_id', timestamp_ix='week')
proc.add_filter(MinUsersPerItem(10, item_ix='article_id', user_ix='customer_id'))
proc.add_filter(MinItemsPerUser(10, item_ix='article_id', user_ix='customer_id'))

transaction_matrix = proc.process(transactions)

  0%|          | 0/1264753 [00:00<?, ?it/s]

  0%|          | 0/1264753 [00:00<?, ?it/s]

In [15]:
knn = TARSItemKNN(K=580, fit_decay=0.1, predict_decay=1/3, similarity='cosine')
# knn = ItemKNN(K=90,normalize_X=False,normalize_sim=True,pop_discount=None,similarity='cosine') 
knn.fit(transaction_matrix)

2022-11-22 16:31:43,283 - base - recpack - INFO - Fitting TARSItemKNN complete - Took 9.65s


TARSItemKNN(K=580, fit_decay=0.1, predict_decay=0.3333333333333333)

In [16]:
prediction_matrix = knn.predict(transaction_matrix)

In [17]:
def top_n_idx_sparse(matrix: csr_matrix, n: int) -> list[list]:
    """Return index of top n values in each row of a sparse matrix.
    source: https://stackoverflow.com/questions/49207275/finding-the-top-n-values-in-a-row-of-a-scipy-sparse-matrix
    """
    top_n_idx = []
    for le, ri in zip(matrix.indptr[:-1], matrix.indptr[1:]):
        n_row_pick = min(n, ri - le)
        top_n_idx.append(matrix.indices[le + np.argpartition(matrix.data[le:ri], -n_row_pick)[-n_row_pick:]].tolist())
    return top_n_idx

def get_top_k_similar_articles_per_user(prediction_matrix: csr_matrix, interaction_matrix: InteractionMatrix, k: int) -> pd.DataFrame:
    """given a prediction matrix and a transaction matrix, return a dataframe with the top k similar articles per user
    """
    # use interaction_matrix._df to map back to original customer and article ids
    uid_cid_map = interaction_matrix._df[["uid", "customer_id"]].drop_duplicates().set_index("uid").to_dict()["customer_id"]
    iid_aid_map = interaction_matrix._df[["iid", "article_id"]].drop_duplicates().set_index("iid").to_dict()["article_id"]

    # get column indices of top k articles per user
    top_k_idx = top_n_idx_sparse(prediction_matrix, k)

    similar_customers = []
    similar_articles = []

    for i, row in enumerate(top_k_idx):
        user_predictions = [iid_aid_map[iid] for iid in row]
        similar_customers.extend([uid_cid_map[i]] * len(user_predictions))
        similar_articles.extend(user_predictions)
        
    assert len(similar_customers) == len(similar_articles), "lengths of lists should be equal"
    return pd.DataFrame({"customer_id": similar_customers, "article_id": similar_articles})

In [18]:
# get top 12 similar articles per user
similar_items = get_top_k_similar_articles_per_user(prediction_matrix, transaction_matrix, k=12)

In [19]:
test_set_transactions.head()

,t_dat,customer_id,sales_channel_id,week
0,2020-07-08,857913002275398,1,104
1,2020-07-08,1658289241058394,1,104
2,2020-07-08,3828854365940846,1,104
3,2020-07-08,4195624216542755,1,104
4,2020-07-08,4233235614030232,2,104


In [20]:
candidates_similar_items = pd.merge(
    similar_items,
    test_set_transactions,
    on='customer_id',
    how='left'
)

candidates_similar_items.head()

,customer_id,article_id,t_dat,sales_channel_id,week
0,857913002275398,599580038,2020-07-08,1,104
1,857913002275398,730863043,2020-07-08,1,104
2,857913002275398,636207014,2020-07-08,1,104
3,857913002275398,720504010,2020-07-08,1,104
4,857913002275398,865595002,2020-07-08,1,104


In [21]:
# get the last price of all article_id's
last_price = transactions \
    .groupby(['article_id', 'week']) \
    .price \
    .last() \
    .reset_index() \
    .groupby('article_id') \
    .price \
    .last() \
    .reset_index()
    
last_price.head()

,article_id,price
0,108775015,0.002068
1,108775044,0.008458
2,110065001,0.006763
3,110065002,0.005186
4,110065011,0.011847


In [22]:
candidates_similar_items = pd.merge(
    candidates_similar_items,
    last_price,
    on='article_id',
    how='left'
)

candidates_similar_items.head()

,customer_id,article_id,t_dat,sales_channel_id,week,price
0,857913002275398,599580038,2020-07-08,1,104,0.016932
1,857913002275398,730863043,2020-07-08,1,104,0.049932
2,857913002275398,636207014,2020-07-08,1,104,0.033881
3,857913002275398,720504010,2020-07-08,1,104,0.033881
4,857913002275398,865595002,2020-07-08,1,104,0.015356


### Time-weighted popularity candidates

In [23]:
popularity = transactions\
    .groupby(['article_id', 'week']).size().reset_index(name='weekly_purchase_count')\

popularity.head()

,article_id,week,weekly_purchase_count
0,108775015,95,2
1,108775015,96,1
2,108775044,94,17
3,108775044,95,3
4,108775044,96,8


In [24]:
weekly_popularity = []
def func(row):
    weeks_before = popularity[(row.article_id == popularity.article_id) & (row.week > popularity.week)]
    # get last row of weeks_before
    previous_week_popularity = 0
    if weeks_before.shape[0] > 0:
        previous_week_popularity = weekly_popularity[-1]
    return previous_week_popularity / 2.0 + float(row.weekly_purchase_count)

In [25]:
# iterate over all rows
for i, row in popularity.iterrows():
    weekly_popularity.append(func(row))
popularity['weekly_popularity'] = weekly_popularity

In [26]:
popular_articles_per_week = popularity.sort_values(['week', 'weekly_popularity'], ascending=False)\
    .groupby('week').head(20).reset_index(drop=True)

In [27]:
popular_articles_previous_week = pd.merge(popular_articles_per_week, mean_price, on=['week', 'article_id']).reset_index(drop=True)
popular_articles_previous_week.week += 1

In [28]:
unique_transactions = transactions \
    .groupby(['week', 'customer_id']) \
    .head(1) \
    .drop(columns=['article_id', 'price']) \
    .copy()

In [29]:
candidates_most_popular = pd.merge(
    unique_transactions,
    popular_articles_previous_week,
    on='week',
)

In [30]:
test_set_transactions = unique_transactions.drop_duplicates('customer_id').reset_index(drop=True)
test_set_transactions.week = test_week

In [31]:
candidates_most_popular_test_week = pd.merge(
    test_set_transactions,
    popular_articles_previous_week,
    on='week'
)
candidates_most_popular_test_week.head()

,t_dat,customer_id,sales_channel_id,week,article_id,weekly_purchase_count,weekly_popularity,price
0,2020-07-08,857913002275398,1,104,909370001,1283,1307.000000,0.032947
1,2020-07-08,857913002275398,1,104,751471001,607,1295.238281,0.033268
2,2020-07-08,857913002275398,1,104,918292001,546,1104.414062,0.041424
3,2020-07-08,857913002275398,1,104,915526001,443,1090.375000,0.033136
4,2020-07-08,857913002275398,1,104,706016001,453,1083.681641,0.033259


In [32]:
candidates_most_popular = pd.concat([candidates_most_popular, candidates_most_popular_test_week])
candidates_most_popular.drop(columns=['weekly_purchase_count', 'weekly_popularity'], inplace=True)
candidates_most_popular.head()

,t_dat,customer_id,sales_channel_id,week,article_id,price
0,2020-07-15,272412481300040,1,95,806388001,0.013301
1,2020-07-15,272412481300040,1,95,730683021,0.025643
2,2020-07-15,272412481300040,1,95,610776002,0.008303
3,2020-07-15,272412481300040,1,95,805308002,0.013609
4,2020-07-15,272412481300040,1,95,866383006,0.024971


# Combining transactions and candidates / negative examples

In [33]:
transactions['purchased'] = 1

In [34]:
data = pd.concat([transactions, candidates_last_purchase, candidates_similar_items])
data.purchased.fillna(0, inplace=True)

data.purchased.mean()

0.43696144574490225

In [35]:
data.drop_duplicates(['customer_id', 'article_id', 'week'], inplace=True)

In [36]:
data = pd.merge(data, articles, on='article_id', how='left')
data = pd.merge(data, customers, on='customer_id', how='left')

### Add item popularity information

In [37]:
data = pd.merge(
    data,
    popular_articles_previous_week[['week', 'article_id', 'weekly_purchase_count', 'weekly_popularity']],
    on=['week', 'article_id'],
    how='left'
)
data['weekly_purchase_count'].fillna(0, inplace=True)
data['weekly_popularity'].fillna(0, inplace=True)
data.head()

,t_dat,customer_id,article_id,price,sales_channel_id,week,purchased,product_code,prod_name,product_type_no,...,garment_group_name,detail_desc,FN,Active,club_member_status,fashion_news_frequency,age,postal_code,weekly_purchase_count,weekly_popularity
0,2020-07-08,857913002275398,599580068,0.008458,1,94,1.0,599580,36,59,...,12,52,1,1,0,1,22,289682,0.0,0.0
1,2020-07-08,857913002275398,776237011,0.025407,1,94,1.0,776237,34,298,...,12,296,1,1,0,1,22,289682,0.0,0.0
2,2020-07-08,857913002275398,844294001,0.011847,1,94,1.0,844294,21663,298,...,12,17410,1,1,0,1,22,289682,0.0,0.0
3,2020-07-08,1658289241058394,877773001,0.007610,1,94,1.0,877773,11114,265,...,8,9636,-1,-1,0,0,21,0,0.0,0.0
4,2020-07-08,3828854365940846,507883009,0.013542,1,94,1.0,507883,1299,286,...,4,-1,-1,-1,0,0,50,105519,0.0,0.0


### preprocessing

In [38]:
from sklearn.preprocessing import StandardScaler, LabelEncoder
scaler = StandardScaler()
columns_to_scale = ['product_type_no', 'graphical_appearance_no', 'colour_group_code', 'perceived_colour_value_id',
'perceived_colour_master_id', 'department_no', 'index_code',
'index_group_no', 'section_no', 'garment_group_no', 'FN', 'Active',
'club_member_status', 'fashion_news_frequency', 'age', 'postal_code', 'weekly_purchase_count', 'weekly_popularity']
data[columns_to_scale] = scaler.fit_transform(data[columns_to_scale])

In [39]:
data.sort_values(['week', 'customer_id'], inplace=True)
data.reset_index(drop=True, inplace=True)

In [40]:
train = data[data.week != test_week]
test = data[data.week==test_week].drop_duplicates(['customer_id', 'article_id', 'sales_channel_id']).copy()

In [41]:
train_baskets = train.groupby(['week', 'customer_id'])['article_id'].count().values

In [42]:
columns_to_use = ['article_id', 'product_type_no', 'graphical_appearance_no', 'colour_group_code', 'perceived_colour_value_id',
'perceived_colour_master_id', 'department_no', 'index_code',
'index_group_no', 'section_no', 'garment_group_no', 'FN', 'Active',
'club_member_status', 'fashion_news_frequency', 'age', 'postal_code', 'weekly_popularity']

In [43]:
%%time

train_X = train[columns_to_use]
train_y = train['purchased']

test_X = test[columns_to_use]

CPU times: user 58 ms, sys: 89.5 ms, total: 147 ms
Wall time: 176 ms


In [44]:
train_y.head()

0    1.0
1    1.0
2    1.0
3    1.0
4    1.0
Name: purchased, dtype: float64

# Model training

In [45]:
from lightgbm.sklearn import LGBMRanker

In [46]:
ranker = LGBMRanker(
    objective="lambdarank",
    num_leaves=200,
    metric="ndcg",
    boosting_type="dart",
    n_estimators=100,
    importance_type='gain',
    verbose=10,
)

In [47]:
%%time

ranker = ranker.fit(
    train_X,
    train_y,
    group=train_baskets,
)

[LightGBM] [Debug] Dataset::GetMultiBinFromSparseFeatures: sparse rate 0.984202
[LightGBM] [Debug] Dataset::GetMultiBinFromAllFeatures: sparse rate 0.109356
[LightGBM] [Debug] init for col-wise cost 0.017577 seconds, init for row-wise cost 0.069318 seconds
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.052223 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Debug] Using Dense Multi-Val Bin
[LightGBM] [Info] Total Bins 1273
[LightGBM] [Info] Number of data points in the train set: 3623509, number of used features: 18
[LightGBM] [Debug] Trained a tree with leaves = 200 and depth = 18
[LightGBM] [Debug] Trained a tree with leaves = 200 and depth = 19
[LightGBM] [Debug] Trained a tree with leaves = 200 and depth = 17
[LightGBM] [Debug] Trained a tree with leaves = 200 and depth = 18
[LightGBM] [Debug] Trained a tree with leaves = 200 and depth = 18
[Ligh

In [48]:
for i in ranker.feature_importances_.argsort()[::-1]:
    print(columns_to_use[i], ranker.feature_importances_[i]/ranker.feature_importances_.sum())

article_id 0.34912614780997553
product_type_no 0.16912234344651791
department_no 0.1274249419564104
garment_group_no 0.07363612751705581
colour_group_code 0.0620460039786994
graphical_appearance_no 0.05047625619943867
section_no 0.04366720179501966
perceived_colour_master_id 0.043078008323887806
perceived_colour_value_id 0.03353592367986232
weekly_popularity 0.018149845759264982
index_code 0.013946483883497503
index_group_no 0.010185739159339664
age 0.004485994704658704
postal_code 0.0008981496070880179
club_member_status 0.00016633675561841138
fashion_news_frequency 3.9713243352570494e-05
Active 1.4782180312610702e-05
FN 0.0


# Calculate predictions

In [49]:
%time

test['preds'] = ranker.predict(test_X)

c_id2predicted_article_ids = test \
    .sort_values(['customer_id', 'preds'], ascending=False) \
    .groupby('customer_id')['article_id'].apply(list).to_dict()

bestsellers_last_week = \
    bestsellers_previous_week[bestsellers_previous_week.week == bestsellers_previous_week.week.max()]['article_id'].tolist()

CPU times: user 1e+03 ns, sys: 1 µs, total: 2 µs
Wall time: 3.1 µs


# Evaluate results

In [50]:
val_week_purchases_by_cust = pd.read_pickle(f'{DATA_PATH}/val_week_purchases_by_cust.pkl')

In [51]:
apks = []

for c_id, gt in val_week_purchases_by_cust.items():
    pred = c_id2predicted_article_ids.get(c_id, [])
    pred = pred + bestsellers_last_week
    apks.append(apk(gt, pred[:12]))

np.mean(apks) # 0.02309837248864059

0.022999555945080668